In [1]:
%gui qt

In [2]:
#import libraries
from sfepy.discrete.fem import FEDomain, Field, domain
#from sfepy.discrete.fem import Mesh
import numpy as nm

In [3]:
from sfepy.mesh.mesh_generators import gen_block_mesh
# creates mesh
dim = [250, 25] #dimension of the block
shape_1 = [250, 5] # nodes in each direction
center = [dim[0]/2, dim[1]/2] # center of teh block
#center = [0,0]
#center = [0,dim[1]/2]
mesh = gen_block_mesh(dim, shape_1, center) # generate mesh and the element connectivity

sfepy: generating 1250 vertices...
sfepy: ...done
sfepy: generating 996 cells...
sfepy: ...done


In [4]:
print(mesh)

Mesh:block
  cmesh:
    CMesh: n_coor: 1250, dim 2, tdim: 2, n_el 996
  descs:
    list: ['2_4']
  dim:
    2
  dims:
    list: [2]
  io:
    None
  n_el:
    996
  n_nod:
    1250
  name:
    block
  nodal_bcs:
    dict with keys: []


In [5]:
#create region geometry
domain = FEDomain('domain', mesh) # create the domain
omega = domain.create_region('Omega', 'all') #create region

In [6]:
R = domain.create_region('L', 'vertices in x > %f' % (dim[0] - 0.001), 'facet') # right surface of the beam
L = domain.create_region('R', 'vertices in x < 0.001', 'facet') # left surface of the bea.
P = domain.create_region('P', 'vertex 4','vertex') # the node point where the load is going to applied
# vertex 1 refers to top left corner

In [7]:
field = Field.from_args('fu', nm.float64, 'vector', omega, approx_order=1) # setup for the vector field

In [8]:
#setting up the field and test variables
from sfepy.discrete import (FieldVariable, Material, Integral, Function, Equation, Equations, Problem)
u = FieldVariable('u','unknown', field)
v = FieldVariable('v','test', field, primary_var_name='u')

In [9]:
# material constants for Asphult
young = 2400.0 # Young's modulus [MPa]
poisson = 0.4  # Poisson's ratio
# setting up the material property and order of inegration
from sfepy.mechanics.matcoefs import stiffness_from_youngpoisson
m = Material('m', values = {'D' : stiffness_from_youngpoisson(2, young, poisson)})
f = Material('Load', values={'.val' : [0.0, -1.0]}) # here it is point load in MPa
integral = Integral('i', order = 2) # defined order of the gauss integral order

In [10]:
# Integral terms

from sfepy.terms import Term
t1 = Term.new('dw_lin_elastic(m.D, v, u)',integral, omega, m=m, v=v,u=u) # linear elastic term
t2 = Term.new('dw_point_load(Load.val, v)',integral,P,Load=f,v=v) # point load term

In [11]:
# the balance equation of t1 = t2
eq = Equation('balance', t1-t2)
eqs = Equations([eq])

In [12]:
# essential boundary condition
from sfepy.discrete.conditions import Conditions, EssentialBC
fix_t = EssentialBC('fix_b', R, {'u.all' : 0.0})

In [13]:
# setup of solution envournment
from sfepy.base.base import IndexedStruct
from sfepy.solvers.ls import ScipyDirect, ScipyUmfpack
from sfepy.solvers.nls import Newton

ls = ScipyDirect({}) # Scipy direct is used to emply a minimization alogorithm
nls_status = IndexedStruct() # is to show the structure



In [14]:
nls = Newton({'i_max' : 10,}, lin_solver=ls, status=nls_status) 
# Newton's method of csolving a nonlinear system with maximum iterations being 10
ebcs = Conditions([fix_t]) # comining all the boindary conditions
type(ebcs)

sfepy.discrete.conditions.Conditions

In [15]:
# solve the problem
pb = Problem('balance', equations=eqs) # setting the equations for soltion
pb.save_regions_as_groups('regions')
pb.set_bcs(ebcs=ebcs)
pb.set_solver(nls) # solver is set to be Newton's linear solver 
status = IndexedStruct()
state,val = pb.solve(status=status, save_results=False)
pb.save_state('sloution_1.vtk', state = state, file_per_var=False)

sfepy: saving regions as groups...
sfepy:   Omega
sfepy:   L
sfepy:   R
sfepy:   P
sfepy: ...done
sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (2490, 2490)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 38740 (6.25e-01% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     Load
sfepy:     m
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 1.000000e+00 (rel: 1.000000e+00)
sfepy:   residual:    0.01 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.00 [s]
sfepy: nls: iter: 1, residual: 3.063280e-11 (rel: 3.063280e-11)
sfepy: solved in 1 steps in 0.02 seconds


In [16]:
import os
from mayavi.core.api import Engine
from mayavi.sources.vtk_file_reader import VTKFileReader
from mayavi.modules.surface import Surface
from mayavi import mlab
mlab.clf()
vtkFile = 'sloution_1.vtk' # feeding mayavi the solution vector

# Create the MayaVi engine and start it.
engine = Engine()
engine.start()
scene = engine.new_scene()

# Read in VTK file and add as source
reader = VTKFileReader()
reader.initialize(vtkFile)
engine.add_source(reader)

In [17]:
from numpy import array

# Adding axes to the visualizer
from mayavi.modules.axes import Axes
axes = Axes()
vtk_file_reader = engine.scenes[0].children[0]
engine.add_filter(axes, vtk_file_reader)

# creating the vase surface for the referance
from mayavi.modules.surface import Surface
surface = Surface()
module_manager = engine.scenes[0].children[0].children[0]
engine.add_filter(surface, module_manager)

#add vector module color coding and others
from mayavi.modules.vectors import Vectors
vectors = Vectors()
engine.add_filter(vectors, module_manager)

In [18]:
#application od legand
module_manager = engine.scenes[0].children[0].children[0]
module_manager.vector_lut_manager.show_scalar_bar = True
module_manager.vector_lut_manager.show_legend = True

In [19]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [20]:
# code to show the co-ordinates of nodes
import meshio
mesh = meshio.read('sloution_1.vtk')
co_ords = mesh.points
l = len(co_ords)
n = l+1
print(l)
#Creation of an array using Range() Function
import numpy as np
m = np.arange(0,n-1)
arr = np.array(m)
print(type(co_ords),type(m))
q = numpy.reshape(co_ords, (1250,3))
w = np.c_[arr,q]
w

1250
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 6.25000000e+00, 0.00000000e+00],
       [2.00000000e+00, 0.00000000e+00, 1.25000000e+01, 0.00000000e+00],
       [3.00000000e+00, 0.00000000e+00, 1.87500000e+01, 0.00000000e+00],
       [4.00000000e+00, 0.00000000e+00, 2.50000000e+01, 0.00000000e+00],
       [5.00000000e+00, 1.00401606e+00, 0.00000000e+00, 0.00000000e+00],
       [6.00000000e+00, 1.00401606e+00, 6.25000000e+00, 0.00000000e+00],
       [7.00000000e+00, 1.00401606e+00, 1.25000000e+01, 0.00000000e+00],
       [8.00000000e+00, 1.00401606e+00, 1.87500000e+01, 0.00000000e+00],
       [9.00000000e+00, 1.00401606e+00, 2.50000000e+01, 0.00000000e+00],
       [1.00000000e+01, 2.00803213e+00, 0.00000000e+00, 0.00000000e+00],
       [1.10000000e+01, 2.00803213e+00, 6.25000000e+00, 0.00000000e+00],
       [1.20000000e+01, 2.00803213e+00, 1.25000000e+01, 0.00000000e+00],
       [1.30000000e+01, 2.00803213e+00, 1.87500000e

In [21]:
#print the displacement variable 
print(u)

FieldVariable:u
  _order:
    0
  _variables:
    Variables
  adof_conns:
    dict with keys: [('u', 'Omega', 'volume', False), ('u', 'P', 'point', False)]
  data:
    list: [array([ 1.19081992e-01, -1.60057766e+00,  5.94948336e-02, -1.60060226e+00,
           -3.52759840e-06, -1.60075674e+00, -5.95400664e-02, -1.60119982e+00,
           -1.20104471e-01, -1.60272696e+00,  1.19082855e-01, -1.59100169e+00,
            5.95023251e-02, -1.59103446e+00,  2.29542504e-05, -1.59118562e+00,
           -5.94600914e-02, -1.59163731e+00, -1.19949693e-01, -1.59243367e+00,
            1.19084940e-01, -1.58142557e+00,  5.95077385e-02, -1.58145584e+00,
            5.54292867e-05, -1.58161243e+00, -5.94199860e-02, -1.58196226e+00,
           -1.19846588e-01, -1.58243234e+00,  1.19085859e-01, -1.57184528e+00,
            5.95132029e-02, -1.57187276e+00,  8.29012782e-05, -1.57201541e+00,
           -5.94022977e-02, -1.57226134e+00, -1.19763002e-01, -1.57257299e+00,
            1.19085490e-01, -1.56226185